In [ ]:
!pip install arxiv pymupdf

In [ ]:
import arxiv
import pandas as pd

# Define your query and settings
search = arxiv.Search(
    query="cat:cs.AI",  # Change to cs.CV or any other category
    max_results=100, # Reduced max_results to a smaller number
    sort_by=arxiv.SortCriterion.SubmittedDate
)

client = arxiv.Client()

# Collect metadata
results = []
for result in client.results(search):
    results.append({
        "Title": result.title,
        "Authors": ", ".join([author.name for author in result.authors]),
        "Date": result.published.date(),
        "Summary": result.summary.strip().replace("\n", " "),
        "pdf_url": result.pdf_url
    })

    print

# Display as a table
df = pd.DataFrame(results)
df.to_csv("data.csv", index=False)

print(df)

In [ ]:
#code for downloading function
import os, requests

os.makedirs("downloads", exist_ok=True)

for result in results:
    title = result["Title"].replace(' ', '_').replace('/','_')
    pdf_url = result["pdf_url"]
    response = requests.get(pdf_url)


    if response.status_code == 200:
      with open(f"downloads/{title}.pdf", "wb") as f:
        f.write(response.content)
    else:
      print(f"Failed to download {title}")

In [ ]:
#extracting content from the fetched documents
import fitz

def extract_text(pdf_path):
  text = ""
  doc = fitz.open(pdf_path)

  for page in doc:
    text += page.get_text()

  return text

file_paths = [...] # add filepath of the downloaded doucments

docs_content = []

for file_path in file_paths:
  docs_content.append(extract_text(file_path))

In [ ]:
# extracting chunks
def chunk_doucuments(docs, chunk_size=1):
  chunks = []
  for doc in docs:
    paragraphs = [p.strip() for p in doc.split("\n") if p.strip()]
    chunks.extend(paragraphs)

  return chunks

abstracts = [result['Summary'] for result in results]
chunks = chunk_doucuments(abstracts)

print(chunks, f"\n{len(chunks)}")